In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("mertrain.csv")
test = pd.read_csv("mertest.csv")

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [12]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X = train.drop(['y'], axis=1).values
X_test= test.values

In [13]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(X)

5

In [14]:
for train_index, test_index in kf.split(X):
    X_train, y_train=X[train_index], y[train_index]
    X_valid, y_valid=X[test_index], y[test_index]

In [23]:
param_grid={"alpha":[0.01,0.1,1.0,10,100],
            "normalize":[False, True]
           }

In [24]:
estimator = Ridge(fit_intercept=True)

In [25]:
clf= GridSearchCV(estimator, param_grid, scoring='r2', cv=kf, n_jobs =2)

In [26]:
clf.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
       error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'alpha': [0.01, 0.1, 1.0, 10, 100], 'normalize': [False, True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [28]:
clf.best_params_

{'alpha': 10, 'normalize': False}

In [29]:
ridge=Ridge(alpha=10, fit_intercept=True, normalize=False)

In [30]:
selector = RFECV(estimator, step=5, cv=5, n_jobs=2)

In [31]:
selector.fit(X_train, y_train)

RFECV(cv=5,
   estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
   n_jobs=2, scoring=None, step=5, verbose=0)

In [32]:
selector.n_features_

52

In [33]:
print("R^2 valid: {: }".format(r2_score(y_valid, selector.predict(X_valid))))
print("R^2 train: {: }".format(r2_score(y_train, selector.predict(X_train))))

R^2 valid:  0.5513893684591542
R^2 train:  0.5443523331388729
